In [2]:
import xml.etree.ElementTree as Xet
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings('ignore')

In [20]:
# Read in the GeoJSON file
kansas_2022 = './ks_2022g (1)/ks_2022g.shp'
gdf_kansan_2022 = gpd.read_file(kansas_2022)
gdf_kansan_2022.head(5)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,...,53,66,56,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42..."
1,001,000020,Cottage Grove Township Voting District,19,108,43,1,171,72,16,...,48,29,49,36,42,33,45,44,34,"POLYGON Z ((-95.46656 37.75217 0.00000, -95.46..."
2,001,000030,Deer Creek Township Voting District,11,65,17,1,94,44,7,...,16,28,19,29,18,30,17,31,15,"POLYGON Z ((-95.33574 37.98185 0.00000, -95.33..."
3,001,00006A,Humboldt Township Voting District,29,127,53,2,211,104,15,...,42,60,43,59,43,63,39,61,38,"POLYGON Z ((-95.47322 37.84735 0.00000, -95.47..."
4,001,170040,Iola Ward 4,169,366,303,8,846,210,119,...,93,228,94,232,90,226,93,224,93,"POLYGON Z ((-95.40017 37.90415 0.00000, -95.40..."


In [52]:
len(gdf_kansan_2022)

4379

In [4]:
gdf_kansan_2022.columns

Index(['COUNTYFP', 'VTDST', 'NAMELSAD', 'REG_DEM', 'REG_REP', 'REG_UNA',
       'REG_LIB', 'REG_TOTAL', 'G22USSRMOR', 'G22USSDHOL', 'G22USSLGRA',
       'G22GOVDKEL', 'G22GOVRSCH', 'G22GOVIPYL', 'G22GOVLCOR', 'G22SOSRSCH',
       'G22SOSDREP', 'G22SOSLLAN', 'G22ATGRKOB', 'G22ATGDMAN', 'G22TRERJOH',
       'G22TREDROG', 'G22TRELROB', 'G22INSRSCH', 'G22INSDCOR', 'G22SSC1YES',
       'G22SSC1NO', 'G22SSC2YES', 'G22SSC2NO', 'G22SSC3YES', 'G22SSC3NO',
       'G22SSC5YES', 'G22SSC5NO', 'G22SSC6YES', 'G22SSC6NO', 'G22SSC7YES',
       'G22SSC7NO', 'geometry'],
      dtype='object')

Still need: Kansas senate, Kansas House of representatives, Constitutional Amendment Rules and Regulations, Sheriff Election and Recall, Turnout information?

### Kansas Senate: state senates are elected at year 2020, 2024..., not in 2022 general election

In [8]:

senate_csv_path = './2022-General-Election-Kansas-Senate-Results-By-Precinct.xlsx'

all_sheets = pd.read_excel(senate_csv_path, sheet_name=None)

print("Sheet names:", all_sheets.keys())

for sheet_name, df in all_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    display(df.head()) 


Sheet names: dict_keys(['KS Senate'])
Processing sheet: KS Senate


,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Clark,Appleton Township,Kansas Senate 38,"Lara, Jose",Democratic,30,000010
1,Clark,Appleton Township,Kansas Senate 38,"Ryckman Sr, Ronald W.",Republican,282,000010
2,Clark,Center 1,Kansas Senate 38,"Lara, Jose",Democratic,27,000020
3,Clark,Center 1,Kansas Senate 38,"Ryckman Sr, Ronald W.",Republican,189,000020
4,Clark,Center 2,Kansas Senate 38,"Lara, Jose",Democratic,18,000030
...,...,...,...,...,...,...,...
231,Seward,Seward Township 3,Kansas Senate 38,"Ryckman Sr, Ronald W.",Republican,15,000210
232,Seward,Liberal Township Exclave 2,Kansas Senate 38,"Lara, Jose",Democratic,0,190010
233,Seward,Liberal Township Exclave 2,Kansas Senate 38,"Ryckman Sr, Ronald W.",Republican,0,190010
234,Seward,Liberal Township Exclave 3,Kansas Senate 38,"Lara, Jose",Democratic,0,190020


### Read in county fips for better matching

In [68]:

county_fips_path = './US_FIPS_Codes (1).xls'

county_fip = pd.read_excel(county_fips_path)
kansas_fip = county_fip[county_fip['U.S. FIPS County Codes'] == 'Kansas']

kansas_fip.drop(columns=['U.S. FIPS County Codes', 'Unnamed: 2'], inplace=True)
kansas_fip.rename(columns={'Unnamed: 3': 'CountyFP'}, inplace=True)
kansas_fip.rename(columns={'Unnamed: 1': 'COUNTYNA'}, inplace=True)
kansas_fip


,COUNTYNA,CountyFP
886,Allen,001
887,Anderson,003
888,Atchison,005
889,Barber,007
890,Barton,009
...,...,...
986,Washington,201
987,Wichita,203
988,Wilson,205
989,Woodson,207


The original PBER only has a County FP column, so I add a county name column to PBER

In [69]:
merged_df = pd.merge(gdf_kansan_2022, kansas_fip, left_on='COUNTYFP', right_on='CountyFP', how='outer', indicator=True)
merged_df.head(5)


,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,CountyFP,_merge
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,...,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42...",Allen,001,both
1,001,000020,Cottage Grove Township Voting District,19,108,43,1,171,72,16,...,36,42,33,45,44,34,"POLYGON Z ((-95.46656 37.75217 0.00000, -95.46...",Allen,001,both
2,001,000030,Deer Creek Township Voting District,11,65,17,1,94,44,7,...,29,18,30,17,31,15,"POLYGON Z ((-95.33574 37.98185 0.00000, -95.33...",Allen,001,both
3,001,00006A,Humboldt Township Voting District,29,127,53,2,211,104,15,...,59,43,63,39,61,38,"POLYGON Z ((-95.47322 37.84735 0.00000, -95.47...",Allen,001,both
4,001,170040,Iola Ward 4,169,366,303,8,846,210,119,...,232,90,226,93,224,93,"POLYGON Z ((-95.40017 37.90415 0.00000, -95.40...",Allen,001,both


In [80]:
pber_county_counts = merged_df['COUNTYNA'].value_counts().reset_index()
pber_county_counts.columns = ['COUNTYNA', 'Count']
pber_county_counts

,COUNTYNA,Count
0,Johnson,610
1,Sedgwick,518
2,Shawnee,248
3,Wyandotte,144
4,Douglas,116
...,...,...
100,Comanche,5
101,Wallace,5
102,Stanton,3
103,Wichita,3


### Kansas House

The election result is an excel file with 5 different sheet. The first sheet Kansas2022 has results for 101 counites, and 4 other sheets have results for the other 4 counties with a large number of precincts

In [13]:
house_csv_path = './2022-General-Election-Kansas-House-of-Representatives-Results-By-Precinct.xlsx'

all_sheets = pd.read_excel(house_csv_path, sheet_name=None)

print("Sheet names:", all_sheets.keys())

dfs = {}  # Dictionary to store each DataFrame

for sheet_name, df in all_sheets.items():
    dfs[sheet_name] = df  # Assign each DataFrame to the dictionary

    print(f"Processing sheet: {sheet_name}")
    display(df) 


Sheet names: dict_keys(['KSHouse', 'JOHNSON', 'SEDGWICK', 'SHAWNEE', 'WYANDOTTE'])
Processing sheet: KSHouse


,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Allen,Carlyle Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,26.0,000010
1,Allen,Carlyle Township,Kansas House of Representatives 9,"Gardner, Fred",Republican,101.0,000010
2,Allen,Cottage Grove Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,19.0,000020
3,Allen,Cottage Grove Township,Kansas House of Representatives 9,"Gardner, Fred",Republican,67.0,000020
4,Allen,Deer Creek Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,10.0,000030
...,...,...,...,...,...,...,...
4278,Woodson,Toronto Township,Kansas House of Representatives 13,"Newland, Joe",Republican,169.0,000070
4279,Woodson,Yates Center Ward 1,Kansas House of Representatives 13,"Newland, Joe",Republican,203.0,00008A
4280,Woodson,Yates Center Ward 1 Exclave,Kansas House of Representatives 13,"Newland, Joe",Republican,0.0,00008B
4281,Woodson,Yates Center Ward 2,Kansas House of Representatives 13,"Newland, Joe",Republican,198.0,000090


Processing sheet: JOHNSON


,PRECINCT CODE,PRECINCT NAME,VTD,Unnamed: 3,Kansas Representative 5th District,Kansas Representative 5th District.1,Kansas Representative 8th District,Kansas Representative 8th District.1,Kansas Representative 8th District.2,Kansas Representative 14th District,...,Kansas Representative 78th District.1,Kansas Representative 78th District.2,Kansas Representative 108th District,Kansas Representative 108th District.1,Kansas Representative 117th District,Kansas Representative 117th District.1,Kansas Representative 117th District.2,Kansas Representative 121st District,Kansas Representative 121st District.1,Kansas Representative 121st District.2
0,NaN,NaN,NaN,NaN,Carrie Barth,Write-in,Chris Croft,Pam Shernuk,Write-in,Charlotte Esau,...,W. Michael Shimeall,Write-in,Brandon Woodard,Write-in,Courtney Tripp,Adam Turk,Write-in,Mel Pinick,John Resman,Write-in
1,1,Aubry Township Precinct 01,10,1001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Aubry Township Precinct 02,20,1002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Aubry Township Precinct 03,220010,1003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Aubry Township Precinct 04,40,1004,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,607,Westwood Precinct 02,3150,R002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,608,Westwood Hills Precinct 01,3130,S001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
609,609,Lake Quivira Precinct 01,190,T001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
610,610,Bonner Springs Ward 4 Precinct 01,900030,U401,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Processing sheet: SEDGWICK


,PRECINCT CODE,PRECINCT NAME,State Representative 72nd District,State Representative 81st District,State Representative 82nd District,State Representative 82nd District.1,State Representative 83rd District,State Representative 84th District,State Representative 85th District,State Representative 85th District.1,...,State Representative 97th District,State Representative 98th District,State Representative 99th District,State Representative 100th District,State Representative 100th District.1,State Representative 101st District,State Representative 103rd District,State Representative 103rd District.1,State Representative 105th District,State Representative 105th District.1
0,NaN,NaN,REP Avery Anderson,REP Blake Carpenter,DEM Misti Hobbs,REP Leah Howell,DEM Henry Helgerson,DEM Ford Carr,DEM Jalon Britton,REP Patrick A. Penn,...,REP Nick Hoheisel,REP Cyndi Howerton,REP Susan Humphries,DEM Mike McCorkle,REP Daniel Hawkins,REP Joe Seiwert,DEM Angela Martinez,LIB Loren John Hermreck,DEM Jaelynn Elise Abegg,REP Brenda K. Landwehr
1,1,101,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,102,0,0,0,0,470,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,84,34,0,0
4,4,104,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,30,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,361,WA09,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
362,362,WA11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
363,363,WA12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364,364,WA13,0,0,0,0,0,0,0,0,...,13,0,0,0,0,0,0,0,0,0


Processing sheet: SHAWNEE


,PRECINCT CODE,PRECINCT NAME,For State Representative 50th District,For State Representative 50th District.1,For State Representative 52nd District,For State Representative 52nd District.1,For State Representative 52nd District.2,For State Representative 53rd District,For State Representative 53rd District.1,For State Representative 53rd District.2,...,For State Representative 54th District.2,For State Representative 55th District,For State Representative 55th District.1,For State Representative 55th District.2,For State Representative 56th District,For State Representative 56th District.1,For State Representative 57th District,For State Representative 57th District.1,For State Representative 58th District,For State Representative 58th District.1
0,NaN,NaN,REP Fred C. Patton,Write-in,DEM Derik Flerlage,REP Jesse Borjon,Write-in,DEM Kirk R. Haskins,REP Bruce Williamson,Write-in,...,Write-in,DEM Tobias Schlingensiepen,REP Todd C. Staerkel,Write-in,DEM Virgil Weigel,Write-in,DEM John Alcala,Write-in,DEM Vic Miller,Write-in
1,1,1 East Rossville,324,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2 West Rossville,282,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,9 East Silver Lake,432,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,10 West Silver Lake,295,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1501,Ward 15 Precinct 1,0,0,0,0,0,0,0,0,...,0,0,0,0,320,31,0,0,0,0
262,1502,Ward 15 Precinct 2,0,0,0,0,0,0,0,0,...,0,0,0,0,239,10,0,0,0,0
263,1503,Ward 15 Precinct 3,0,0,0,0,0,0,0,0,...,0,0,0,0,32,0,0,0,0,0
264,1504,Ward 15 Precinct 4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Processing sheet: WYANDOTTE


,PRECINCT CODE,PRECINCT NAME,For State Representative 31st District,For State Representative 31st District.1,For State Representative 31st District.2,For State Representative 32nd District,For State Representative 32nd District.1,For State Representative 33rd District,For State Representative 33rd District.1,For State Representative 33rd District.2,...,For State Representative 35th District.1,For State Representative 35th District.2,For State Representative 36th District,For State Representative 36th District.1,For State Representative 36th District.2,For State Representative 37th District,For State Representative 37th District.1,For State Representative 37th District.2,For State Representative 38th District,For State Representative 38th District.1
0,NaN,NaN,REP Dennis Grindel,DEM Louis E. Ruiz,Write-in,DEM Pam Curtis,Write-in,LIB Stephanie Barton,DEM Bill Hutton,REP Mike Thompson,...,REP Sam Stillwell,Write-in,REP Kevin Braun,DEM Lynn Melton,Write-in,DEM Melissa Oropeza,REP Diana Whittington,Write-in,REP Timothy H. Johnson,Write-in
1,1,BS01-01,0,0,0,0,0,9,228,340,...,0,0,0,0,0,0,0,0,0,0
2,2,BS02-01,0,0,0,0,0,6,159,268,...,0,0,0,0,0,0,0,0,0,0
3,3,BS03-01,0,0,0,0,0,19,268,321,...,0,0,0,0,0,0,0,0,0,0
4,4,BS04-01,0,0,0,0,0,19,413,446,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,KC14-15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,326,10
146,146,KC14-16,0,0,0,0,0,0,0,0,...,0,0,104,166,0,0,0,0,0,0
147,147,KC14-17,0,0,0,0,0,0,0,0,...,0,0,235,323,0,0,0,0,0,0
148,148,QC01-01,0,0,0,0,0,0,11,9,...,0,0,0,0,0,0,0,0,0,0


Count how many precints each county has.

In [58]:
KSHouse = dfs['KSHouse']
JOHNSON_COUNTY = dfs['JOHNSON']
SEDGWICK_COUNTY = dfs['SEDGWICK']
SHAWNEE_COUNTY = dfs['SHAWNEE']
WYANDOTTE_COUNTY = dfs['WYANDOTTE']

In [59]:
unique_combinations = KSHouse[['County', 'Precinct']].drop_duplicates()
len(unique_combinations)

3144

In [79]:
election_county_counts = KSHouse['County'].value_counts().reset_index()
election_county_counts.columns = ['County', 'Count']
election_county_counts

,County,Count
0,Douglas,194
1,Riley,178
2,Miami,167
3,Reno,147
4,Montgomery,138
...,...,...
96,Grant,6
97,Wallace,5
98,Stanton,3
99,Greeley,3


In [61]:
unique_combinations1 = JOHNSON_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations2 = SEDGWICK_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations3 = SHAWNEE_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations4 = WYANDOTTE_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()

len(unique_combinations1) + len(unique_combinations2) + len(unique_combinations3) + len(unique_combinations4)

1394

The total number of precincts from Official Kansas election result is 3144 + 1394 = 4538, which is different from 4379 from PBER from IKE Lab. This means that additional investigations have to be made.

Create a table to display the differences in the number of counites for the 2 datasets.

In [82]:
pd.set_option('display.max_rows', None)

merged_counts = pd.merge(pber_county_counts, election_county_counts, left_on='COUNTYNA', right_on='County', how='outer', indicator=True)

merged_counts


,COUNTYNA,Count_x,County,Count_y,_merge
0,Johnson,610,NaN,NaN,left_only
1,Sedgwick,518,NaN,NaN,left_only
2,Shawnee,248,NaN,NaN,left_only
3,Wyandotte,144,NaN,NaN,left_only
4,Douglas,116,Douglas,194.0,both
5,Saline,94,Saline,134.0,both
6,Reno,86,Reno,147.0,both
7,Montgomery,84,Montgomery,138.0,both
8,Riley,81,Riley,178.0,both
9,Miami,75,Miami,167.0,both


### Process 104 counties

In [62]:
candidate_votes = KSHouse.groupby(['Candidate', 'Party', 'District'])['Votes'].sum().reset_index()

candidate_votes.rename(columns={'Votes': 'Total_Votes'}, inplace=True)

candidate_votes_sorted = candidate_votes.sort_values(by='District')

candidate_votes_sorted

,Candidate,Party,District,Total_Votes
40,"Houser, Michael",Republican,1,5847.0
64,"Rogers, Paul D.",Democratic,1,1928.0
34,"Haswood, Christina",Democratic,10,6695.0
72,"Seiwert, Joe",Republican,101,1187.0
60,"Probst, Jason",Democratic,102,2719.0
...,...,...,...,...
58,"Pickert, Sandy",Republican,88,254.0
68,"Schmidt, Chuck",Democratic,88,129.0
30,"Gardner, Fred",Republican,9,6101.0
17,"Cloutier, Alana",Democratic,9,2127.0


Using a similar approach to create column names and pivot

In [84]:
election_results_cleaned = KSHouse

# Extract district number from the 'Race' column and create new column
election_results_cleaned['District'] = election_results_cleaned['Race'].str.extract(r'(\d+)', expand=False)

# Function to create a unique column name based on the office, district, candidate, and party
def create_column_name(row):
    office_abbr = "SL"  # Abbreviation for State Legislature
    party_initial = row['Party'][0]
    last_name = row['Candidate'].split()[0][:3].upper()
    district = row['District'].zfill(3)  # Ensure the district number has leading zeros
    
    # Create a unique column name that includes office, district, candidate, and party
    unique_name = f"{office_abbr}{district}{party_initial}{last_name}"
    
    return unique_name

# Apply the function to generate a unique column name for each candidate
election_results_cleaned['ColumnName'] = election_results_cleaned.apply(create_column_name, axis=1)

# Pivot dataset so that each column represents a candidate (unique column name) and each row represents a precinct
pivoted_df = election_results_cleaned.pivot_table(
    index=['County', 'Precinct', 'VTD'],
    columns='ColumnName',
    values='Votes',
    aggfunc='sum'
)

# Flatten the columns
pivoted_df.columns = [col for col in pivoted_df.columns]

# Reset index
pivoted_df.reset_index(inplace=True)

# Display the final DataFrame
pivoted_df.head(5)

,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005RBAR,SL006DFRI,...,SL116RHOF,SL117DTRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA
0,Allen,Bassett City Enclave,00020C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allen,Carlyle Township,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allen,Cottage Grove Township,000020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allen,Deer Creek Township,000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allen,East Elm,000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
matching_combinations = pd.merge(
    merged_df[['COUNTYNA', 'VTDST']], 
    pivoted_df[['County', 'VTD']],    
    left_on=['COUNTYNA', 'VTDST'],     
    right_on=['County', 'VTD'],        
    how='inner'                        
)

# Count the number of matching combinations
num_matching_combinations = len(matching_combinations)
num_matching_combinations

2371

There are only 2371 out of 3144 precincts that match, indicating there needs more processing

Check that after pivoting, the numbers match previous results(they did)

In [88]:
def check_total_votes_after_pivot(df, original_df):
    df.reset_index(inplace=True)

    candidate_totals = {}
    
    for column in df.columns[3:]:  # Start after 'County' and 'Precinct'
        candidate_totals[column] = df[column].sum()

    print("\nTotal Votes for Each Candidate by Office After Pivoting:")
    for candidate, total_votes in candidate_totals.items():
        print(f"{candidate}: {total_votes}")
    
    print("\nValidation against original DataFrame:")
    original_totals = original_df.groupby('ColumnName')['Votes'].sum()
    print(original_totals)


check_total_votes_after_pivot(pivoted_df, election_results_cleaned)


Total Votes for Each Candidate by Office After Pivoting:
VTD: 00020C00001000002000003000004000005000006A00006B19002000006D00006C00007000008019001017001017002017003017003B17003C17004000013000014000015000016A00016C00017000018000019000020A00020B00021000001000002A00002B00002C00003000004000005000006000008020001020002000010000011000012000013000014A00016000017000018000019000020000021000022000023000024000001000002000003000004000005A00006A00006B00006C00007A00007B00007C00008A00008B18001000008D18002000009012003000010000011000012000013000014010005000016A00017A00017B00001000002000003000004000005000006000007000008000009000010000012B00011000012A00013000014000015000016000017000018000019000020000021000022000023000001000002000003000004000005000006000007000008000009000010000011A00012000013000014000015000016000017000018000019A00019B00020000021000022A00023C00023A00023B00024000025000026000027A00028022001022000000030000031000032000033000034000035000036000037000038000039000001000002A00002B00002C00002D00002H0

### Rules and Regulations

In [7]:
rules_csv_path = './2022-General-Election-Constitutional-Amendment-Question-1-Rules-and-Regulations-Results-By-Precinct.xlsx'
rules_results = pd.read_excel(rules_csv_path)
rules_results

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Allen,Carlyle Township,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""YES""",NaN,73.0,000010
1,Allen,Carlyle Township,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""NO""",NaN,52.0,000010
2,Allen,Cottage Grove Township,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""YES""",NaN,56.0,000020
3,Allen,Cottage Grove Township,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""NO""",NaN,30.0,000020
4,Allen,Deer Creek Township,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""YES""",NaN,19.0,000030
...,...,...,...,...,...,...,...
6293,Woodson,Yates Center Ward 1 Exclave,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""NO""",NaN,0.0,00008B
6294,Woodson,Yates Center Ward 2,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""YES""",NaN,112.0,000090
6295,Woodson,Yates Center Ward 2,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""NO""",NaN,90.0,000090
6296,Woodson,Yates Center Ward 2 Exclave,Constitutional Amendment - Legislative Oversig...,"Oversight, Legislative - ""YES""",NaN,0.0,900010
